In [1]:
import numpy as np
import tensorflow as tf

#importar los datos:
#importar desde 3 archivos diferentes, labels x,y x archivo.
#x_train,y_train = datostrain.x,datostrain.y
#x_validation, y_validation = datosvalidation.x, datosvalidation.y
#x_testing, y_testing = datostesting.x,datostesting.y
x_train=np.array([[1.0, 0.0, 2.0, 3.0, 0.0, 1.0, 1.0, 3.0, 4.0],[0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0]],dtype=np.float32)
batch_size=32


In [2]:
def import_data(file):
    file = open(file,'r')
    train_data = []
    train_labels = []
    file.readline()
    for line in file:
        data = line.split(';')[1][1:-1].split(',')
        for x in data:
            train_data.append(float(x))
            train_labels.append(line.split(';')[2])
    
    train_data = tf.convert_to_tensor(train_data)
    train_labels = tf.convert_to_tensor(train_labels)

    train_data = tf.reshape(train_data, [-1,1,200,1])  
  # print(len(train_data))
  # print(train_data)
    train_labels = tf.reshape(train_labels,[-1,1])
    return train_data,train_labels

In [3]:
train_data, train_labels = import_data("database/database.csv")
print(train_data)

Tensor("Reshape:0", shape=(21926, 1, 200, 1), dtype=float32)


In [4]:
# Dimensiones arreglo de data. 
N,_,D,_ = train_data.shape
# Redimensionar para usar tensores de Nx1 y Dx1

# Input de la red, None es para utilzar cualquier numero de input
x_input = tf.placeholder(tf.float32, shape=[batch_size,1,D.value,1], name='x-input')

# Output de la red
target = tf.placeholder(tf.float32, name='target', shape=(batch_size, 10))

# filtro
filter_size, n_filters = 5, 12
# Son n_filters filtros de tamaño filter_size


In [5]:
# Model blocks
def conv_layer(input_tensor, kernel_shape, layer_name):
    # input_tensor b01c
    # kernel_shape 01-in-out
    weights = tf.get_variable("weights", kernel_shape,
                               initializer = tf.contrib.layers.xavier_initializer_conv2d())
    print(weights)
    biases = tf.get_variable("biases", [kernel_shape[3]],
                             initializer=tf.constant_initializer(0.05))
    

    
    # Other options are to use He et. al init. for weights and 0.01 
    # to init. biases.
    conv = tf.nn.conv2d(input_tensor, weights, 
                       strides = [1, 1, 1, 1], padding='SAME')
    return tf.nn.relu(conv + biases)

def fc_layer(input_tensor, weights_shape, layer_name):
    # weights_shape in-out
    weights = tf.get_variable("weights", weights_shape,
                              initializer = tf.contrib.layers.xavier_initializer())
    biases = tf.get_variable("biases", [weights_shape[1]],
                             initializer=tf.constant_initializer(0.0))
    tf.summary.histogram(layer_name + "/weights", weights)
    tf.summary.histogram(layer_name + "/biases", biases)
    mult_out = tf.matmul(input_tensor, weights)
    return tf.nn.relu(mult_out+biases)

In [8]:


layer_name='conv1'
with tf.variable_scope(layer_name):
    conv1_out = conv_layer(x_input, [1, filter_size, 1, n_filters], layer_name)
with tf.name_scope('pool1'):
    pool1_out = tf.nn.max_pool(conv1_out, ksize=[1, 1, 1, 1],
                          strides=[1, 1, 2, 1], padding='SAME',
                          name='pool1')
layer_name='conv2'
with tf.variable_scope(layer_name):
    conv2_out = conv_layer(pool1_out, [1,filter_size,n_filters,n_filters], layer_name)
with tf.name_scope('pool2'):
    pool2_out = tf.nn.max_pool(conv2_out, ksize=[1, 1, 1, 1],
                          strides=[1, 1, 2, 1], padding='SAME',
                          name='pool2')

print(pool2_out.shape)

<tf.Variable 'conv1/weights:0' shape=(1, 5, 1, 12) dtype=float32_ref>
<tf.Variable 'conv2/weights:0' shape=(1, 5, 12, 12) dtype=float32_ref>
(32, 1, 50, 12)


In [9]:
print(pool2_out.shape)
pool2_out_flat = tf.reshape(pool2_out, [-1, 1*50*12], name='pool2_flat')
print(pool2_out_flat.shape)

(32, 1, 50, 12)
(32, 600)


In [11]:
# First fully connected layer
keep_prob = tf.placeholder(tf.float32, name='dropout_prob', shape=())

layer_name = 'fc1'
with tf.variable_scope(layer_name):
    fc1_out = fc_layer(pool2_out_flat, [1*50*12, 300], layer_name)
fc1_out_drop = tf.nn.dropout(fc1_out, keep_prob)

ValueError: Variable fc1/weights already exists, disallowed. Did you mean to set reuse=True in VarScope? Originally defined at:

  File "<ipython-input-5-b37e338c32c0>", line 22, in fc_layer
    initializer = tf.contrib.layers.xavier_initializer())
  File "<ipython-input-10-744376d81cc4>", line 6, in <module>
    fc1_out = fc_layer(pool2_out_flat, [1*50*12, 300], layer_name)
  File "/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py", line 2910, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)


In [12]:
# Second fully connected layer
layer_name = 'fc2'
with tf.variable_scope(layer_name):
    fc2_out = fc_layer(fc1_out_drop, [300, 10], layer_name)
model_output = fc2_out

In [13]:
print(model_output.shape)

(32, 10)


In [14]:
with tf.name_scope('loss_function'):
    cross_entropy = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits=model_output, labels=target,
                                           name='cross_entropy'))
    tf.summary.scalar('cross_entropy', cross_entropy)
with tf.name_scope('optimizer1'):
    optimizer = tf.train.AdamOptimizer()
    grads_vars = optimizer.compute_gradients(cross_entropy)
    optimizer.apply_gradients(grads_vars)
    train_step = optimizer.minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(model_output, 1),
                             tf.argmax(target, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name='accuracy')

In [ ]:
def validate():
    batches = cifar10.getValidationSet(asBatches=True)
    accs = []
    for batch in batches:
        data, labels = batch
        acc = sess.run(accuracy,
                       feed_dict={
                model_input: data,
                target: labels,
                keep_prob: 1.0
            })
        accs.append(acc)
    mean_acc = np.array(accs).mean()
    summary = sess.run(merged,
                       feed_dict={
                model_input: data,
                target: labels,
                keep_prob: 1.0
            })
    return summary, mean_acc
def test():
    batches = cifar10.getTestSet(asBatches=True)
    accs = []
    for batch in batches:
        data, labels = batch
        acc = sess.run(accuracy,
                       feed_dict={
                model_input: data,
                target: labels,
                keep_prob: 1.0
            })
        accs.append(acc)
    mean_acc = np.array(accs).mean()
    return mean_acc
